In [ ]:
########### Basic Parameters for Running: ################################
    
TFliteNamingAndVersion = "knd"      # Used for tflite Filename
Training_Percentage = 0.2           
Epoch_Anz = 100
nb_classes = 2                     # move to 1. step
input_shape = (32, 20,3)
Batch_Size = 32

##########################################################################


In [ ]:
from bn.dataloading import tmnist_percentals
from bn.plot_functions import plot_dataset

x_train, y_train = tmnist_percentals()

plot_dataset(x_train, y_train)

In [ ]:
from bn.class_encoding import sin_cos_encoding
        
y_train = sin_cos_encoding(y_train)

In [ ]:
import numpy as np
from bn.augmentation import augmentation
from bn.plot_functions import plot_dataset_it
from sklearn.model_selection import train_test_split

# Split train and validation data 
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=Training_Percentage)

train_iterator = augmentation(x_train, y_train)
validation_iterator = augmentation(x_test, y_test)

plot_dataset_it(train_iterator)


In [ ]:
from  bn.models.effnet import Effnet
import tensorflow as tf

model = Effnet(input_shape, nb_classes, activation_top=None)

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              optimizer="adam", metrics = ["accuracy"])

In [ ]:
from bn.plot_functions import plot_acc_loss

history = model.fit(train_iterator, 
                validation_data=validation_iterator, 
                batch_size=Batch_Size, 
                epochs = 15,
                verbose=1)

plot_acc_loss(history, "Trainingsergebnis")

In [ ]:
from bn.class_encoding import sin_cos_encoding_revert
import numpy as np
from bn.plot_functions import plot_test_divergence
from operator import itemgetter

res = []
stat_Anz = []
stat_Abweichung = []
false_img = []
false_label = []
i = 0
deviation_max_list = 0.05
test_count = tc = 1000

for i in range(100):
    stat_Anz.append(0)
    stat_Abweichung.append(0)

for x_test, y_test in validation_iterator:
    if (tc<0):
        break
    tc = tc - Batch_Size
           
    for (x, y) in zip(x_test, y_test):
            
        target_sin = y[0]
        target_cos = y[1]
        target = sin_cos_encoding_revert(y)
        classes = model.predict(x.reshape(1,32,20,3))
        
        out_sin = classes[0][0]  
        out_cos = classes[0][1]
        out_target = sin_cos_encoding_revert(classes[0])

        dev_sin = target_sin - out_sin
        dev_cos = target_cos - out_cos
        dev_target = target - out_target
        if abs(dev_target + 1) < abs(dev_target):
            out_target = out_target - 1
            dev_target = target - out_target
        else:
            if abs(dev_target - 1) < abs(dev_target):
                out_target = out_target + 1
                dev_target = target - out_target
                
        target_int = int ((target) * 10)
        stat_Abweichung[target_int] = stat_Abweichung[target_int] + dev_target  
        stat_Anz[target_int] = stat_Anz[target_int] + 1
                
        res.append([target, out_target, dev_target, out_sin, out_cos, i])
        if abs(dev_target) > deviation_max_list:
            lbl_text = "Expected: " + str(round(target*10,1)) + "\n Predicted: " + str(round(out_target*10,1)) +  "\n diff: " + str(round(dev_target*10,1))
            #print("Exp: " + str(round(target*10,1)) + " Pred: " + str(round(out_target*10,1)) +  " diff: " + str(round(dev_target*10,1)))
            false_img.append(x)
            false_label.append(lbl_text)
    
for i in range(100):
    if (stat_Anz[i] != 0):
        stat_Abweichung[i] = stat_Abweichung[i] / stat_Anz[i]

res = np.asarray(res)
res_step_1 = res
print(f"False Predicted: {len(false_label)}, accurace on images= {1-(len(false_label)/test_count)}" )
plot_dataset(np.array(false_img).astype(np.uint8), false_label, columns=11, rows=3)
res1 = np.array(sorted(res, key=itemgetter(0)))
plot_test_divergence(res1[:,3], res1[:,4])


In [1]:
from PIL import Image 
import glob
import os

def ziffer_data_files():
    Input_dir='images'
    imgfiles = []
    for root, dirs, files in os.walk(Input_dir):
        for file in files:
            if (file.endswith(".jpg") and not file.startswith("10_")):
                print(root + "/" + file)
                imgfiles.append(root + "/" + file)
    return  imgfiles

def ziffer_data(x_data, y_data, nb_classes):

    files = ziffer_data_files()
    print(len(files))

    for aktfile in files:
        base = os.path.basename(aktfile)
        target = base[0:1]
        if target == "N":
            category = 10                # NaN does not work --> convert to 10

        else:
            category = int(target)
        test_image = Image.open(aktfile).resize((20, 32))
        test_image = np.array(test_image, dtype="float32")

        # if only 10 classes, ignore the category 10
        if (nb_classes>10 or category<10):
            x_data.append(test_image)
            y_data.append(np.array([category]))
    return x_data, y_data

xz_test, yz_test = ziffer_data([],[], 11)

images/watermeter1/20220503/03/3_dig4_20220503-031836.jpg
images/watermeter1/20220503/03/0_dig2_20220503-032818.jpg
images/watermeter1/20220503/03/2_dig3_20220503-034251.jpg
images/watermeter1/20220503/03/0_dig2_20220503-035233.jpg
images/watermeter1/20220503/03/3_dig4_20220503-034742.jpg
images/watermeter1/20220503/03/3_dig4_20220503-032327.jpg
images/watermeter1/20220503/03/0_dig1_20220503-035233.jpg
images/watermeter1/20220503/03/0_dig1_20220503-032818.jpg
images/watermeter1/20220503/03/0_dig2_20220503-033309.jpg
images/watermeter1/20220503/03/0_dig2_20220503-030403.jpg
images/watermeter1/20220503/03/0_dig1_20220503-030403.jpg
images/watermeter1/20220503/03/0_dig1_20220503-033309.jpg
images/watermeter1/20220503/03/0_dig1_20220503-030854.jpg
images/watermeter1/20220503/03/2_dig3_20220503-034742.jpg
images/watermeter1/20220503/03/2_dig3_20220503-032327.jpg
images/watermeter1/20220503/03/3_dig4_20220503-034251.jpg
images/watermeter1/20220503/03/0_dig2_20220503-030854.jpg
images/waterme

NameError: name 'np' is not defined

In [ ]:
from PIL import Image 
import glob
import os

def ziffer_data_files():
    Input_dir='images'
    imgfiles = []
    for root, dirs, files in os.walk(Input_dir):
        for file in files:
            if (file.endswith(".jpg") and not file.startswith("10_")):
                print(root + "/" + file)
                imgfiles.append(root + "/" + file)
    return  imgfiles

def ziffer_data(x_data, y_data, nb_classes):

    files = ziffer_data_files()
    print(len(files))

    for aktfile in files:
        base = os.path.basename(aktfile)
        target = base[0:1]
        if target == "N":
            category = 10                # NaN does not work --> convert to 10

        else:
            category = int(target)
        test_image = Image.open(aktfile).resize((20, 32))
        test_image = np.array(test_image, dtype="float32")

        # if only 10 classes, ignore the category 10
        if (nb_classes>10 or category<10):
            x_data.append(test_image)
            y_data.append(np.array([category]))
    return x_data, y_data

xz_test, yz_test = ziffer_data([],[], 11)

images/watermeter1/20220503/03/3_dig4_20220503-031836.jpg
images/watermeter1/20220503/03/0_dig2_20220503-032818.jpg
images/watermeter1/20220503/03/2_dig3_20220503-034251.jpg
images/watermeter1/20220503/03/0_dig2_20220503-035233.jpg
images/watermeter1/20220503/03/3_dig4_20220503-034742.jpg
images/watermeter1/20220503/03/3_dig4_20220503-032327.jpg
images/watermeter1/20220503/03/0_dig1_20220503-035233.jpg
images/watermeter1/20220503/03/0_dig1_20220503-032818.jpg
images/watermeter1/20220503/03/0_dig2_20220503-033309.jpg
images/watermeter1/20220503/03/0_dig2_20220503-030403.jpg
images/watermeter1/20220503/03/0_dig1_20220503-030403.jpg
images/watermeter1/20220503/03/0_dig1_20220503-033309.jpg
images/watermeter1/20220503/03/0_dig1_20220503-030854.jpg
images/watermeter1/20220503/03/2_dig3_20220503-034742.jpg
images/watermeter1/20220503/03/2_dig3_20220503-032327.jpg
images/watermeter1/20220503/03/3_dig4_20220503-034251.jpg
images/watermeter1/20220503/03/0_dig2_20220503-030854.jpg
images/waterme

NameError: name 'np' is not defined

In [ ]:
#np.array(xz_test).shape
for x, y in zip(xz_test, yz_test):
    pred = sin_cos_encoding_revert(model.predict(np.array(xz_test))[0])*10
    if (pred != y):
        print(f"Pred: {pred} , Label: {y}")

In [ ]:
Input_dir='images'
for root, dirs, files in os.walk(Input_dir):
     for file in files:
        if (file.endswith(".jpg") and not file.startswith("10_")):
            print(root + "/" + file)# + '---' + file)